In [ ]:
# Ensure the process is reproducible
set.seed(2)

In [ ]:

# Placeholder for R Data Frame Auto-code


In [ ]:
if(!require(caret)) install.packages('caret')
if(!require(randomForest)) install.packages('randomForest')
if(!require(rpart)) install.packages('rpart')
if(!require(rpart)) install.packages('rpart.plot')
if(!require(rpart)) install.packages('e1071')

In [ ]:
library(randomForest)
library(caret)
library(rpart)
library(rpart.plot)
library(e1071)

In [ ]:
# Create index
trainIndex <- createDataPartition(y= custDataRaw$CHURN, p=0.8, list = FALSE)
 # 80% data goes in here
train <- custDataRaw[trainIndex,]
# 20% data goes in here
test  <- custDataRaw[-trainIndex,] 

In [ ]:
nrow(train)
nrow(test)

In [ ]:
summary(custDataRaw)

In [ ]:
fitCART <- rpart(as.character(CHURN) ~ Gender + Status + Children + Est.Income +
                 Car.Owner + Age + LongDistance + International + Local +
                 Dropped + Paymethod + LocalBilltype + LongDistanceBilltype +
                 Usage + RatePlan,
             data = train,
             method="class")

In [ ]:
rpart.plot(fitCART)

Each node shows
- the predicted class (died or survived),
- the predicted probability of survival,
- the percentage of observations in the node.

In [ ]:
prediction <- predict(fitCART,test,type="class")

In [ ]:
# finalResults <- data.frame(Actual = test$CHURN, Predicted = prediction)
finalResults <- data.frame(Actual = test$CHURN, Predicted = prediction)

In [ ]:
head(finalResults)

In [ ]:
confusionMatrix(prediction, test$CHURN)

In [ ]:
fitRandomForests <- randomForest(as.factor(CHURN) ~ Gender + Status + Children + Est.Income +
                    Car.Owner + Age + LongDistance + International + Local +
                    Dropped + Paymethod + LocalBilltype + LongDistanceBilltype +
                    Usage + RatePlan,
                    data=custDataRaw,
                    importance=TRUE,
                    ntree=100,
                    mtry=3
                    )

In [ ]:
varImpPlot(fitRandomForests, type=1)

There’s two types of importance measures shown above. The accuracy one tests to see how worse the model performs without each variable, so a high decrease in accuracy would be expected for very predictive variables. The Gini one digs into the mathematics behind decision trees, but essentially measures how pure the nodes are at the end of the tree. Again it tests to see the result if each variable is taken out and a high score means the variable was important.

In [ ]:
plot(fitRandomForests, main=paste("Error Rate vs. # Trees ( mtry =",fitRandomForests$mtry,")"), 
     type="l", 
     col.main="black",
     lwd=2,
     lty=1);
legend("top", colnames(fitRandomForests$err.rate),col=1:4,
       cex=0.8, fill=1:5, lwd=1, bty="n")

In [ ]:
print(fitRandomForests$confusion)